In [1]:
# import library
import re
import string
import nltk
import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)

# Data Manipulation
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("ggplot")
%matplotlib inline


In [2]:
# load data
train_data = pd.read_csv('datasets/train.csv')
test_data = pd.read_csv('datasets/test.csv')
sample_data = pd.read_csv('datasets/sample_submission.csv')

In [3]:
train_data.head(20)

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,Interesting that it only targets S1P-1/5 recep...,fingolimod,2
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder wh...",ocrevus,2
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain ...",gilenya,1
5,a043780c757966243779bf3c0d11bf6eef721971,I can’t give you advice about Lemtrada because...,cladribine,2
6,be5a13376933a7f9bbf8e801c31691092f63260a,Reply posted for JessZidek. Hi Jess Sorry to r...,humira,0
7,08c3c0c702fc97d290204b37798ac62005da5626,Well as expected my Neurologist wants me to st...,gilenya,2
8,8fd3d7ad80791c9343e5cf8a83bd1adf6577d516,Why do you think that FIngolimod was such a mi...,fingolimod,1
9,793c5af7cc8332df17eb602247d886fbd1c80f89,Thank you so much…I’m learning a lot here at G...,tagrisso,2


In [4]:
test_data.head()

,unique_hash,text,drug
0,9e9a8166b84114aca147bf409f6f956635034c08,"256 (previously stable on natalizumab), with 5...",fingolimod
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,On fingolimod and have been since December 201...,fingolimod
2,50b6d851bcff4f35afe354937949e9948975adf7,Apparently it's shingles! :-/ I do have a few ...,humira
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,If the Docetaxel doing once a week x3 weeks th...,tagrisso
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,"CC, Stelara worked in a matter of days for me....",stelara


In [5]:
sample_data.head()

,unique_hash,sentiment
0,9e9a8166b84114aca147bf409f6f956635034c08,0
1,e747e6822c867571afe7b907b51f0f2ca67b0e1a,0
2,50b6d851bcff4f35afe354937949e9948975adf7,0
3,7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae,0
4,8b37d169dee5bdae27060949242fb54feb6a7f7f,0


In [6]:
train_data.text[0]

'Autoimmune diseases tend to come in clusters. As for Gilenya – if you feel good, don’t think about it, it won’t change anything but waste your time and energy. I’m taking Tysabri and feel amazing, no symptoms (other than dodgy color vision, but I’ve had it since always, so, don’t know) and I don’t know if it will last a month, a year, a decade, ive just decided to enjoy the ride, no point in worrying.'

In [7]:
train_data.text[20]

'BMC Ophthalmol. 2019 Jan 8;19(1):8. Intravitreal dexamethasone implants versus intravitreal anti-VEGF treatment in treating patients with retinal vein occlusion: a meta-analysis. Gao L et al. Background : Retinal vein occlusion (RVO) is a common retinal venous disorder that causes vision loss. No specific therapy has been developed. Controversy exists regarding two treatments: intravitreal dexamethasone implants and anti-vascular endothelial growth factor (VEGF). The goal of this study is to compare the effectiveness and safety of dexamethasone implants and anti-VEGF treatment for RVO. Methods : The PubMed, Embase, and Cochrane Library databases were searched for studies comparing dexamethasone implants with anti-VEGF in patients with RVO. Best-corrected visual acuity (BCVA), central subfield thickness (CST), intraocular pressure changes, conjunctival haemorrhage, reduced VA, and macular oedema were extracted from the final included studies. RevMan 5.3 was used to conduct the quantita